# 03 SDK | 01 Chat Completion

## Azure Environment

You can create the necessary instance of Azure OpenAI and deploy an embedding model by executing one of the deployment options [mentioned here](../../create_env/src/AzureCLI/CreateEnv.azcli). To execute the sample code service specific information is needed ([Details and instructions here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.8"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);


string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created...");

Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.8
    DotNetEnv, 2.5.0

OpenAI Client created...
```

## A basic interaction with OpenAI Client

The following cell, demonstrate a basic interaction with OpenAI Client. In this case, the system message is used to provide instructions or settings for the assistant, such as its personality or behavior. The user message is used to provide queries or inputs from the human user. The assistant message is used to provide responses or outputs from the model.

In [ ]:
string sys_prompt = "You are an AI assistance. You extract intention from provided text. You always answer with intention:";
ChatCompletionsOptions simpleOption = new ChatCompletionsOptions();

simpleOption.Messages.Add(new ChatMessage(ChatRole.System, sys_prompt));

simpleOption.Messages.Add(new ChatMessage(ChatRole.User, "I'm not receiving calls on my Samsung Galaxy S22. Can you help?"));

//Request Properties
simpleOption.MaxTokens = 500;
simpleOption.Temperature = 0.0f;
simpleOption.NucleusSamplingFactor = 0.0f;
simpleOption.FrequencyPenalty = 0.7f;
simpleOption.PresencePenalty = 0.7f;
simpleOption.StopSequences.Add("\n"); 


// Call the Model ChatCompletions endpoint
Response<ChatCompletions> simpleResponse = await openAIClient.GetChatCompletionsAsync(
    chatCompletionDeploymentName, 
    simpleOption);

// Get the first choice from the response
ChatCompletions simpleCompletions = simpleResponse.Value;
foreach (ChatChoice chatChoice in simpleCompletions.Choices) {
    Console.WriteLine($"ChatChoiceIndex: {chatChoice.Index}");
    Console.WriteLine($"Content: {chatChoice.Message.Content}");
}

Expected outcome:

```
ChatChoiceIndex: 0
Content: Intention: Request for technical assistance with phone call issue on Samsung Galaxy S22.
```

## Step 2: Compose Chat

In [ ]:
//Define System Prompt
string system = @" 
        You extract intention from provided text. The two intentions you identify are: product information and order status. 
        You answer with a valid JSON string. 
        The JSON string must have the format {""Intention"": ""ProductInformation""} or {""Intention"": ""OrderInfo""}. 
        You don't provide additional information. 
        If you can't identify intention answer with {""Intention"": ""Unknown""}
";

//Compose Chat (Few Shot Learning)
ChatCompletionsOptions chatCompletionsOptions = new ChatCompletionsOptions();

chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.System, system));

chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "I've purchased three weeks ago new training shoes. When will they will be delivered?"));
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.Assistant, "{\"Intention\": \"OrderInfo\"}")); 

chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "Still waiting for the delivery. Any idea when it will arrive? I'm Robert and I'm calling on behalf of company Contoso."));
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.Assistant, "{\"Intention\": \"OrderInfo\"}")); 

chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "Do you have training shoes? If yes, I'm interested in running equipment specifically running shoes."));
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.Assistant, "{\"Intention\": \"ProductInfo\"}")); 

chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "What is the average price for running shoes?"));
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.Assistant, "{\"Intention\": \"ProductInfo\"}")); 

//Request Properties
chatCompletionsOptions.MaxTokens = 500;
chatCompletionsOptions.Temperature = 0.0f;
chatCompletionsOptions.NucleusSamplingFactor = 0.0f;
chatCompletionsOptions.FrequencyPenalty = 0.7f;
chatCompletionsOptions.PresencePenalty = 0.7f;
chatCompletionsOptions.StopSequences.Add("\n"); 

//Choices per prompt
chatCompletionsOptions.ChoiceCount = 2;

Console.WriteLine("ChatCompletionsOptions created...");


Expected outcome:

```
ChatCompletionsOptions created...
```

## Step 3: Call OpenAI ChatCompletion Endpoint

In [ ]:
// Add user input to the chat completions options
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "How much do you charge for a pair of first class running shoes?")); 

// Call the Model ChatCompletions endpoint
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(
    chatCompletionDeploymentName, 
    chatCompletionsOptions);

// Get the first choice from the response
ChatCompletions chatCompletions = response.Value;
foreach (ChatChoice chatChoice in chatCompletions.Choices) {
    Console.WriteLine($"ChatChoiceIndex: {chatChoice.Index}");
    Console.WriteLine($"Content: {chatChoice.Message.Content}");
}



Expected outcome:

```
ChatChoiceIndex: 0
Content: {"Intention": "ProductInfo"}
ChatChoiceIndex: 1
Content: {"Intention": "ProductInfo"}
```

## Step 4: Check additional response values

### Token Usage

Tokens consumed by the `GetChatCompletionsAsync()` call can be retrieved by parsing the raw response from the call as the `"nuget: Azure.AI.OpenAI, 1.0.0-beta.7"` does not provide the information as property yet.

In [ ]:
//Get Raw Response
var rawResponse = response.GetRawResponse();
if (!rawResponse.IsError) {
    //Get Raw Response Content
    JsonElement jsonElement = JsonSerializer.Deserialize<JsonElement>(rawResponse.Content.ToString());
    JsonElement tokenUsage = jsonElement.GetProperty("usage");
    tokenUsage.TryGetProperty("completion_tokens", out JsonElement completionTokens);
    Console.WriteLine($"Completion Tokens: {completionTokens.ToString()}");
    tokenUsage.TryGetProperty("prompt_tokens", out JsonElement promptTokens);
    Console.WriteLine($"Prompt Tokens: {promptTokens.ToString()}");
    tokenUsage.TryGetProperty("total_tokens", out JsonElement totalTokens);
    Console.WriteLine($"Total Tokens: {totalTokens.ToString()}");
}    

Expected outcome:

```
Completion Tokens: <value>
Prompt Tokens: <value>
Total Tokens: <value>
```

### Filter 

If the LLM detects:
- hate
- self harm
- violence
- sexual
content within the provided prompt or in the result it will be filtered and reported

In [ ]:
//Prompt Filter
Console.WriteLine($"Prompt Filter Results");
foreach(PromptFilterResult promptFilterResult in chatCompletions.PromptFilterResults) {
    Console.WriteLine($"PromptIndex: {promptFilterResult.PromptIndex}");
    Console.WriteLine($"\t ContentFilterResults: Hate");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.Hate.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.Hate.Severity}");
    Console.WriteLine($"\t ContentFilterResults: SelfHarm");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.SelfHarm.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.SelfHarm.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Violence");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.Violence.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.Violence.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Sexual");
    Console.WriteLine($"\t\t Filtered: {promptFilterResult.ContentFilterResults.Sexual.Filtered}");
    Console.WriteLine($"\t\t Severity: {promptFilterResult.ContentFilterResults.Sexual.Severity}");
}

//Response Filter
foreach (ChatChoice chatChoice in chatCompletions.Choices) {
    Console.WriteLine($"ChatChoiceIndex: {chatChoice.Index}");
    Console.WriteLine($"\t ContentFilterResults: Hate");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.Hate.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.Hate.Severity}");
    Console.WriteLine($"\t ContentFilterResults: SelfHarm");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.SelfHarm.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.SelfHarm.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Violence");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.Violence.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.Violence.Severity}");
    Console.WriteLine($"\t ContentFilterResults: Sexual");
    Console.WriteLine($"\t\t Filtered: {chatChoice.ContentFilterResults.Sexual.Filtered}");
    Console.WriteLine($"\t\t Severity: {chatChoice.ContentFilterResults.Sexual.Severity}");
}

Expected output

```
Prompt Filter Results
ChatChoiceIndex: 0
	 ContentFilterResults: Hate
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: SelfHarm
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Violence
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Sexual
		 Filtered: False
		 Severity: safe
ChatChoiceIndex: 1
	 ContentFilterResults: Hate
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: SelfHarm
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Violence
		 Filtered: False
		 Severity: safe
	 ContentFilterResults: Sexual
		 Filtered: False
		 Severity: safe
```

## Next Steps

- The result from the result was received as a single object. Streaming is useful in scenario where the response is large, and the client wants to process the response as it is received. [Demo Streaming](./02_ChatCompletionStreaming.ipynb)